In [1]:
import pandas as pd
import numpy as np

In [2]:
menu = pd.read_csv("Menu.csv")

# For menu data

## id cannot be null

In [4]:
menu[menu['id'].isnull()]

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count


## id should be Unique

In [7]:
menu['id'].nunique() == menu.shape[0]

True

## page count should not be null

In [6]:
menu[menu['page_count'].isnull()]

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count


## page count should be positive

In [8]:
menu[menu['page_count'] <= 0]

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count


In [9]:
dish = pd.read_csv("Dish.csv")

# For dish data

## id should not be null

In [10]:
dish[dish['id'].isnull()]

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


## id should be unique

In [11]:
dish['id'].nunique() == dish.shape[0]

True

## menus_appeared or times appeared cannot be null

In [13]:
np.sum(dish['menus_appeared'].isnull())

0

In [14]:
np.sum(dish['times_appeared'].isnull())

0

## menus_appeared for the same dish should be less than or equal to times_appeared (violated!)

In [15]:
dish[dish['menus_appeared'] > dish['times_appeared']]

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
194,208,Luncheon,NaN,19,18,1900,1993,0.65,0.65
676,825,"Rice, Semolina",NaN,1,0,1900,1900,0.00,0.00
905,1082,Caviare,NaN,86,85,1888,1906,0.40,0.50
951,1136,Carta blanca,NaN,8,7,1900,1981,0.00,0.00
1133,1346,Hackley's Sour Mash,NaN,5,4,1900,1900,0.15,0.15
...,...,...,...,...,...,...,...,...,...
423317,515598,Apricot Cômpote,NaN,1,0,0,0,0.00,0.00
423318,515599,Pear Cômpote,NaN,1,0,0,0,0.00,0.00
423319,515600,Guava Cômpote,NaN,1,0,0,0,0.00,0.00
423320,515601,Bilberies Cômpote,NaN,1,0,0,0,0.00,0.00


## The same dish's lowest_price should <= highest_price

In [16]:
dish[dish['lowest_price'] > dish['highest_price']]

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


# MenuPage

In [ ]:
menupage = pd.read_csv('menupage.csv')

## page_number for the same menu should start from 1 and be continuous (this cannot be fixed because some pages might be missing, probably we can deduplicate repeating pages but it's not directly related to our use case)

In [26]:
temp = menupage.groupby('menu_id').agg({'page_number':list}).reset_index()
temp['page_number'] = temp['page_number'].apply(sorted)
def notContinuous(page_number):
    return len(page_number) != max(page_number)
temp['violated'] = temp['page_number'].apply(notContinuous)
temp[temp['violated'] == True]

,menu_id,page_number,violated
5620,21276,"[1.0, 1.0, 2.0, 3.0, 4.0, 5.0]",True
5810,21467,"[1.0, 3.0]",True
6068,21725,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",True
6250,21907,"[1.0, 2.0, 4.0, 5.0, 6.0]",True
6605,22265,"[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0,...",True
...,...,...,...
19186,34897,"[1.0, 7.0]",True
19256,34967,"[1.0, 2.0, 3.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0,...",True
19432,35143,"[1.0, 1.0, 2.0, 2.0]",True
19761,35472,"[1.0, 3.0]",True


# MenuItem

In [20]:
menuitem = pd.read_csv('MenuItem.csv')

## created_at should be <= than updated_at (violation exists)

In [21]:
menuitem[menuitem['created_at'] > menuitem['updated_at']]

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
716,773,3375,NaN,NaN,590.0,2011-04-19 18:40:33 UTC,2011-04-19 18:39:49 UTC,0.328571,0.477144
866,938,5379,NaN,NaN,698.0,2011-04-19 18:59:03 UTC,2011-04-19 18:58:17 UTC,0.117143,0.641237
1715,1852,1389,0.25,NaN,1307.0,2011-04-19 20:57:12 UTC,2011-04-19 20:56:41 UTC,0.132857,0.932696
1721,1858,1389,0.25,NaN,1312.0,2011-04-19 20:59:34 UTC,2011-04-19 20:58:32 UTC,0.657143,0.868813
1873,2034,5106,NaN,NaN,1460.0,2011-04-19 21:53:27 UTC,2011-04-19 21:52:43 UTC,0.228571,0.376473
...,...,...,...,...,...,...,...,...,...
940954,985947,22249,0.10,NaN,219.0,2012-06-25 02:02:51 UTC,2012-06-25 01:55:57 UTC,0.301429,0.466596
941007,986003,64816,0.75,NaN,373011.0,2012-06-25 04:05:33 UTC,2012-06-25 03:58:33 UTC,0.071429,0.108947
941282,986280,64817,0.50,NaN,2001.0,2012-06-25 10:45:45 UTC,2012-06-25 10:38:43 UTC,0.488571,0.574253
941342,986340,64816,0.25,0.4,40390.0,2012-06-25 12:00:48 UTC,2012-06-25 11:54:09 UTC,0.534286,0.386687


# fix: swapping the values of the two columns when violation exists

In [22]:
menuitem['created_at_old'] = menuitem['created_at']
menuitem['updated_at_old'] = menuitem['updated_at']
menuitem['created_at'] = menuitem.apply(lambda x:x['created_at_old'] if x['created_at_old'] <= x['updated_at_old'] else x['updated_at_old'],axis=1)
menuitem['updated_at_at'] = menuitem.apply(lambda x:x['updated_at_old'] if x['created_at_old'] <= x['updated_at_old'] else x['created_at_old'],axis=1)
menuitem.drop(columns=['created_at_old','updated_at_old'],inplace=True)

## Now the violation no longer exists

In [23]:
menuitem[menuitem['created_at'] > menuitem['updated_at']]

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos,updated_at_at
